In [1]:
# !git clone https://github.com/Neerajj9/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow.git

Cloning into 'Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow'...
remote: Enumerating objects: 88, done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (88/88), 3.32 MiB | 8.06 MiB/s, done.


In [1]:
import sys  
sys.path.insert(1, '')
from Utils import *

ModuleNotFoundError: No module named 'Utils'

In [ ]:
from keras import backend as K
import keras
import cv2
#from model.Utils import *
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import *
from keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.models import model_from_json
import matplotlib.pyplot as plt
import os

In [ ]:
#Variable Definition
img_w = 512
img_h = 512
channels = 3
classes = 1
info = 5
grid_w = 16
grid_h = 16

In [ ]:
#import data
#X and Y numpy arrays are created using the Prepocess.py file
X = np.load('/home/nouman/Downloads/training_batch/week5/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/Data/X.npy')
Y = np.load('/home/nouman/Downloads/training_batch/week5/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/Data/Y.npy')

In [ ]:
print(X.shape , Y.shape)

(461, 512, 512, 3) (461, 16, 16, 1, 5)


In [ ]:
X_train , X_val , Y_train , Y_val  = train_test_split(X,Y,train_size = 0.75 , shuffle = True)
X = []
Y = []

In [ ]:
#define model
# input : 512,512,3
# output : 16,16,1,5



def save_model(model):
    model_json = model.to_json()
    with open("/home/nouman/Downloads/training_batch/week5/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/model/text_detect_model.json", "w") as json_file:
        json_file.write(model_json)

        
        
def load_model(strr):        
    json_file = open(strr, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model
    

    
    
def yolo_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    conv = Conv2D(512,(3,3) , activation='relu' , padding='same')(last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(128,(3,3) , activation='relu' , padding='same')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(5,(3,3) , activation='relu' , padding='same')(lr)
    
    final = Reshape((grid_h,grid_w,classes,info))(conv)
    
    model = Model(inp,final)
    
    return model



In [ ]:
#define utils

#optimizer
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

#checkpoint
checkpoint = ModelCheckpoint('/home/nouman/Downloads/training_batch/week5/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/Data/text_detect.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min',period=1)


/home/nouman/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
#define loss function
def yolo_loss_func(y_true,y_pred):
    # y_true : 16,16,1,5
    # y_pred : 16,16,1,5
    l_coords = 5.0
    l_noob = 0.5
    coords = y_true[:,:,:,:,0] * l_coords
    noobs = (-1*(y_true[:,:,:,:,0] - 1)*l_noob)
    p_pred = y_pred[:,:,:,:,0]
    p_true = y_true[:,:,:,:,0]
    x_true = y_true[:,:,:,:,1]
    x_pred = y_pred[:,:,:,:,1]
    yy_true = y_true[:,:,:,:,2]
    yy_pred = y_pred[:,:,:,:,2]
    w_true = y_true[:,:,:,:,3]
    w_pred = y_pred[:,:,:,:,3]
    h_true = y_true[:,:,:,:,4]
    h_pred = y_pred[:,:,:,:,4]
    
    p_loss_absent = K.sum(K.square(p_pred - p_true)*noobs)
    p_loss_present = K.sum(K.square(p_pred - p_true))
    x_loss = K.sum(K.square(x_pred - x_true)*coords)
    yy_loss = K.sum(K.square(yy_pred - yy_true)*coords)
    xy_loss = x_loss + yy_loss
    w_loss = K.sum(K.square(K.sqrt(w_pred) - K.sqrt(w_true))*coords)
    h_loss = K.sum(K.square(K.sqrt(h_pred) - K.sqrt(h_true))*coords)
    wh_loss = w_loss + h_loss
    
    loss = p_loss_absent + p_loss_present + xy_loss + wh_loss
    
    return loss

In [ ]:
#load and save model
input_size = (img_h,img_w,channels)

model = yolo_model(input_size)

print(model.summary())

save_model(model)


2023-03-17 16:59:28.321408: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-17 16:59:28.321507: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-17 16:59:28.321601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nouman-HP-EliteBook-840-G2): /proc/driver/nvidia/version does not exist
2023-03-17 16:59:28.322501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 256, 256, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 256, 256, 32  128         ['Conv1[0][0]']                  
                                )                                                             

                                                                                                  
 block_3_expand (Conv2D)        (None, 128, 128, 14  3456        ['block_2_add[0][0]']            
                                4)                                                                
                                                                                                  
 block_3_expand_BN (BatchNormal  (None, 128, 128, 14  576        ['block_3_expand[0][0]']         
 ization)                       4)                                                                
                                                                                                  
 block_3_expand_relu (ReLU)     (None, 128, 128, 14  0           ['block_3_expand_BN[0][0]']      
                                4)                                                                
                                                                                                  
 block_3_p

                                                                                                  
 block_6_depthwise_BN (BatchNor  (None, 32, 32, 192)  768        ['block_6_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_6_depthwise_relu (ReLU)  (None, 32, 32, 192)  0           ['block_6_depthwise_BN[0][0]']   
                                                                                                  
 block_6_project (Conv2D)       (None, 32, 32, 64)   12288       ['block_6_depthwise_relu[0][0]'] 
                                                                                                  
 block_6_project_BN (BatchNorma  (None, 32, 32, 64)  256         ['block_6_project[0][0]']        
 lization)                                                                                        
          

 lization)                                                                                        
                                                                                                  
 block_10_expand_relu (ReLU)    (None, 32, 32, 384)  0           ['block_10_expand_BN[0][0]']     
                                                                                                  
 block_10_depthwise (DepthwiseC  (None, 32, 32, 384)  3456       ['block_10_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_10_depthwise_BN (BatchNo  (None, 32, 32, 384)  1536       ['block_10_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_10_

 alization)                                                                                       
                                                                                                  
 block_14_expand (Conv2D)       (None, 16, 16, 960)  153600      ['block_13_project_BN[0][0]']    
                                                                                                  
 block_14_expand_BN (BatchNorma  (None, 16, 16, 960)  3840       ['block_14_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_14_expand_relu (ReLU)    (None, 16, 16, 960)  0           ['block_14_expand_BN[0][0]']     
                                                                                                  
 block_14_depthwise (DepthwiseC  (None, 16, 16, 960)  8640       ['block_14_expand_relu[0][0]']   
 onv2D)   

                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 16, 16, 512)  0           ['batch_normalization[0][0]']    
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 16, 128)  589952      ['leaky_re_lu[0][0]']            
                                                                                                  
 dropout_1 (Dropout)            (None, 16, 16, 128)  0           ['conv2d_1[0][0]']               
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 16, 16, 128)  512        ['dropout_1[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 leaky_re_

In [ ]:
model.load_weights('/home/nouman/Downloads/training_batch/week5/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/Data/text_detect.h5')

In [ ]:
model.compile(loss=yolo_loss_func , optimizer=opt , metrics = ['accuracy'])

In [ ]:
hist = model.fit(X_train ,Y_train ,epochs= 10 ,batch_size = 8 , callbacks=[checkpoint] , validation_data=(X_val,Y_val))

Epoch 1/10


2023-03-17 16:59:31.952019: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1085276160 exceeds 10% of free system memory.
2023-03-17 16:59:39.980110: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 402653184 exceeds 10% of free system memory.
2023-03-17 16:59:40.079654: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 402653184 exceeds 10% of free system memory.
2023-03-17 16:59:40.490726: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 405805056 exceeds 10% of free system memory.
2023-03-17 16:59:50.927355: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 405805056 exceeds 10% of free system memory.


 8/22 [=========>....................] - ETA: 3:30 - loss: nan - accuracy: 0.7634

In [ ]:
#model.save_weights('model1/text_detect.h5')

In [ ]:
model = load_model('/home/nouman/Downloads/training_batch/week5/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/model/text_detect_model.json')
model.load_weights('/home/nouman/Downloads/training_batch/week5/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/Data/text_detect.h5')

In [ ]:
def predict_func(model , inp , iou , name):

    ans = model.predict(inp)
    
    #np.save('Results/ans.npy',ans)
    boxes = decode(ans[0] , img_w , img_h , iou)
    
    img = ((inp + 1)/2)
    img = img[0]
    #plt.imshow(img)
    #plt.show()


    for i in boxes:

        i = [int(x) for x in i]

        img = cv2.rectangle(img , (i[0] ,i[1]) , (i[2] , i[3]) , color = (0,255,0) , thickness = 2)

    plt.imshow(img)
    plt.show()
    
    cv2.imwrite(os.path.join('Results' , str(name) + '.jpg') , img*255.0)
    
    

In [ ]:
rand = np.random.randint(0,X_val.shape[0], size = 5)

for i in rand:
    
    predict_func(model , X_val[i:i+1] , 0.5, i)
    

In [ ]:
for i in os.listdir('Test'):
    img = cv2.imread(os.path.join('Test',i))
    img = cv2.resize(img,(512,512))
    img = (img - 127.5)/127.5
    predict_func(model , np.expand_dims(img,axis= 0) , 0.5 , 'sample')

### THE END